# Create Modeling Areas

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-03-19

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Modeling Areas
# Author: Timm Nawrocki
# Created on: 2019-03-19
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Modeling Areas" generates a polygon from an array of x, y coordinates that will serve as the general modeling area for data download and processing. The model area polygon selects intersecting sub-basins (8 digit hydrologic units) from the Watershed Boundaries Dataset to provide form the hydrographic area of influence for the region.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input array
point_array = [[-152.79699, 61.55473],
               [-152.70467, 58.66074],
               [-157.69984, 56.33955],
               [-159.49354, 56.97233],
               [-163.09044, 58.59629],
               [-160.84181, 61.57616]]

# Define input datasets
alaska_boundary = os.path.join(root_directory, 'Data_Input/source_data/boundaries/Alaska_63360_ExcludingSmallIslands.shp')
subbasins = os.path.join(root_directory, 'Data_Input/source_data/inland_waters/NHD_H_02_GDB.gdb/WBD/WBDHU8')

# Define spatial references
geographic_reference = 4326 #GCS WGS 1984
projected_reference = 3338 #NAD 1983 Alaska Albers
transform_method = 'WGS_1984_(ITRF00)_To_NAD_1983'

# Define output feature class
model_area_full = 'SouthwestAlaska_ModelArea_Full'
model_area_clipped = 'SouthwestAlaska_ModelArea_Clipped'
hydrographic_area = 'SouthwestAlaska_HydrographicArea'

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Create Model Area Polygons

In [4]:
# Define function to create model area polygons
def create_model_area(**kwargs):
    """
    Description: creates a polygon feature class with vertices read from an array
    Inputs: 'point_array' -- an array of x, y coordinates
            'input_array' -- an array containing a coastline polygon to clip the model area
            'output_array' -- the output should be an array with a full feature class and a clipped feature class
            'output_path' -- the output path is the directory in which the new feature class is created
            'geographic_reference' -- the ESRI reference number for the geographic reference that matches the point array
            'projected_reference' -- the ESRI reference number for the projected reference of the output datasets
            'transform_method' -- required if the geographic and projected references are in different datums
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    point_array = kwargs['point_array']
    alaska_boundary = kwargs['input_array'][0]
    output_path = kwargs['output_path']
    output_full = os.path.join(output_path, kwargs['output_array'][0])
    output_clipped = os.path.join(output_path, kwargs['output_array'][1])
    geographic_reference = arcpy.SpatialReference(kwargs['geographic_reference'])
    projected_reference = arcpy.SpatialReference(kwargs['projected_reference'])
    transform_method = kwargs['transform_method']
    # Define intermediate datasets
    initial_area = os.path.join(output_path, 'modelArea_initial')
    # Create a new polygon feature class
    print('Inserting vertices to new feature class...')
    arcpy.CreateFeatureclass_management(output_path, 'modelArea_initial', 'POLYGON')
    # Add projection to new feature class
    arcpy.DefineProjection_management(initial_area, geographic_reference)
    # Initiate an insert cursor and insert an array of vertices for a polygon
    with arcpy.da.InsertCursor(initial_area, ["SHAPE@"]) as cursor:
        array = arcpy.Array()
        for point in point_array:
            X = point[0]
            Y = point[1]
            array.add(arcpy.Point(X, Y))
        cursor.insertRow([arcpy.Polygon(array)])
    # Project polygon
    arcpy.Project_management(initial_area, output_full, projected_reference, transform_method)
    # Clip the polygon to the Alaska coastline
    print('Clipping model area to Alaska coastline...')
    arcpy.Clip_analysis(output_full, alaska_boundary, output_clipped)
    # Delete intermediate dataset
    arcpy.Delete_management(initial_area)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created modeling areas...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define input and output arrays
model_area_inputs = [alaska_boundary]
model_area_outputs = [model_area_full, model_area_clipped]

# Create key word arguments
create_model_area_kwargs = {'point_array' : point_array,
                            'input_array' : model_area_inputs,
                            'output_array' : model_area_outputs,
                            'output_path' : arcpy.env.workspace,
                            'geographic_reference' : geographic_reference,
                            'projected_reference' : projected_reference,
                            'transform_method' : transform_method
                           }

# Process the create polygon function with the point array
arcpy_geoprocessing(create_model_area, **create_model_area_kwargs)

SouthwestAlaska_ModelArea_Full already exists and will be overwritten.
SouthwestAlaska_ModelArea_Clipped already exists and will be overwritten.
Inserting vertices to new feature class...
Clipping model area to Alaska coastline...
Successfully created modeling areas...
----------
Succeeded at 2019-04-02 13:10 (Elapsed time: 0:00:13)


## 3. Create Hydrographic Area of Influence

In [6]:
# Define function to create hydrographic area of influence
def create_hydrographic_area(**kwargs):
    """
    Description: creates a polygon feature class with vertices read from an array
    Inputs: 'input_array' -- an array containing the full model area and the 8 digit hydrologic units
            'output_array' -- the output should be an array with a feature class for selected 8 digit hydrologic units
            'projected_reference' -- the ESRI reference number for the projected reference of the output datasets
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    model_area = kwargs['input_array'][0]
    subbasins = kwargs['input_array'][1]
    hydrographic_area = kwargs['output_array'][0]
    projected_reference = arcpy.SpatialReference(kwargs['projected_reference'])
    # Define intermediate datasets
    hydrographic_area_initial = 'hydrographicArea_initial'
    # Select subbasins that intersect the model area
    print('Selecting subbasins in hydrographic area...')
    subbasins_select = arcpy.SelectLayerByLocation_management(subbasins, 'INTERSECT', model_area, '', 'NEW_SELECTION', 'NOT_INVERT')
    # Project polygon
    print('Copying subbasins to new feature class...')
    arcpy.CopyFeatures_management(subbasins_select, hydrographic_area_initial)
    arcpy.Project_management(hydrographic_area_initial, hydrographic_area, projected_reference)
    # Delete intermediate dataset
    arcpy.Delete_management(hydrographic_area_initial)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created hydrographic area of influence...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [7]:
# Define input and output arrays
hydrographic_area_inputs = [model_area_full, subbasins]
hydrographic_area_outputs = [hydrographic_area]

# Create key word arguments
create_hydrographic_area_kwargs = {'input_array' : hydrographic_area_inputs,
                                   'output_array' : hydrographic_area_outputs,
                                   'projected_reference' : projected_reference
                                  }

# Process the create polygon function with the point array
arcpy_geoprocessing(create_hydrographic_area, **create_hydrographic_area_kwargs)

SouthwestAlaska_HydrographicArea already exists and will be overwritten.
Selecting subbasins in hydrographic area...
Copying subbasins to new feature class...
Successfully created hydrographic area of influence...
----------
Succeeded at 2019-04-02 13:10 (Elapsed time: 0:00:04)
